In [1]:
import pandas as pd

In [2]:
dump = pd.read_csv("goes_dump.csv")

In [ ]:
# Process flare casses into dataframe
flare_classes = pd.DataFrame(dump["goes_class"].unique(), columns=["goes_class"])
flare_classes = flare_classes.sort_values("goes_class")
flare_classes.reset_index(drop=True)


,goes_class
0,A6.8
1,A8.9
2,A9.1
3,A9.5
4,A9.6
...,...
300,X7.1
301,X8.2
302,X8.7
303,X9.0


In [ ]:
# Split out subclasses to assign each sequence a class letter and a sub value (intensity).

split_classes = dump.copy()

# Split "goes_class" into letter and number
split_classes[["class_letter", "class_value"]] = split_classes["goes_class"].str.extract(r"([A-Z])([0-9.]+)")
split_classes["class_value"] = split_classes["class_value"].astype(float)

split_classes.head()


,start_time,peak_time,end_time,goes_class,noaa_active_region,class_letter,class_value
0,2010-05-01 01:34:00,2010-05-01 01:39:00,2010-05-01 01:43:00,C5.7,11067,C,5.7
1,2010-05-01 05:23:00,2010-05-01 05:27:00,2010-05-01 05:31:00,B1.6,11064,B,1.6
2,2010-05-01 09:48:00,2010-05-01 09:52:00,2010-05-01 09:59:00,B1.0,11067,B,1.0
3,2010-05-02 06:03:00,2010-05-02 06:09:00,2010-05-02 06:18:00,B2.9,11067,B,2.9
4,2010-05-02 10:02:00,2010-05-02 10:09:00,2010-05-02 10:15:00,B3.8,11067,B,3.8


In [ ]:
import pandas as pd

# Print the class intensity table from the literature.

class_intensity = pd.DataFrame({
    "class": ["A", "B", "C", "M", "X"],
    "intensity W/m^2": [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
})

print(class_intensity)

#class_intensity.to_csv("solar_flare_class_intensity.csv", index=False)


  class  intensity W/m^2
0     A     1.000000e-08
1     B     1.000000e-07
2     C     1.000000e-06
3     M     1.000000e-05
4     X     1.000000e-04


In [ ]:
# Define the class base intensities as a dictionary
class_multipliers = dict(zip(class_intensity["class"], class_intensity["intensity W/m^2"]))

# Compute the intensity using tabe from literature.
split_classes["intensity_W/m^2"] = split_classes["class_letter"].map(class_multipliers) * split_classes["class_value"]

split_classes


,start_time,peak_time,end_time,goes_class,noaa_active_region,class_letter,class_value,intensity_W/m^2
0,2010-05-01 01:34:00,2010-05-01 01:39:00,2010-05-01 01:43:00,C5.7,11067,C,5.7,5.700000e-06
1,2010-05-01 05:23:00,2010-05-01 05:27:00,2010-05-01 05:31:00,B1.6,11064,B,1.6,1.600000e-07
2,2010-05-01 09:48:00,2010-05-01 09:52:00,2010-05-01 09:59:00,B1.0,11067,B,1.0,1.000000e-07
3,2010-05-02 06:03:00,2010-05-02 06:09:00,2010-05-02 06:18:00,B2.9,11067,B,2.9,2.900000e-07
4,2010-05-02 10:02:00,2010-05-02 10:09:00,2010-05-02 10:15:00,B3.8,11067,B,3.8,3.800000e-07
...,...,...,...,...,...,...,...,...
19153,2025-05-31 15:36:00,2025-05-31 15:49:00,2025-05-31 15:57:00,M2.9,14100,M,2.9,2.900000e-05
19154,2025-05-31 18:13:00,2025-05-31 18:20:00,2025-05-31 18:24:00,C2.5,14100,C,2.5,2.500000e-06
19155,2025-05-31 19:00:00,2025-05-31 19:06:00,2025-05-31 19:08:00,C4.8,14100,C,4.8,4.800000e-06
19156,2025-05-31 21:54:00,2025-05-31 21:58:00,2025-05-31 22:02:00,C2.2,14100,C,2.2,2.200000e-06


In [ ]:
# Store data as CSV to merge with parent SHARP dataset.
split_classes.to_csv("flare_events.csv")